파일열기

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel('/content/drive/Othercomputers/내 노트북/study/06_TextAnalytics/data/yelp.xlsx')

문서 단어 행렬 만들기

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, stop_words='english')
dtm = cv.fit_transform(df.review)

x와 y를 **지정**

In [3]:
x = dtm
y = df.sentiment

데이터 분할

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,   # 20%의 데이터를 테스트용으로 유보
    random_state=42) # 유사난수의 씨앗값 seed을 42로 설정

나이브 베이즈 분류

In [5]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
model.fit(x_train, y_train)

BernoulliNB()

훈련 데이터로 정확도(accuracy) 평가

In [6]:
model.score(x_train, y_train)

0.92125

테스트 데이터로 정확도 평가

In [7]:
model.score(x_test, y_test)

0.765

단어별 확률

In [8]:
prob_df = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '비율': model.feature_log_prob_[1] - model.feature_log_prob_[0]
})

상대적으로 긍정 문장에서 많이 나오는 단어

In [9]:
prob_df.sort_values('비율').tail(10)

,단어,비율
26,attentive,1.965811
744,spot,1.965811
221,fantastic,2.099343
365,loved,2.099343
12,amazing,2.159967
472,perfect,2.217126
208,excellent,2.322486
32,awesome,2.504808
153,delicious,3.155395
268,great,4.062952


상대적으로 부정 문장에서 많이 나오는 단어

In [10]:
prob_df.sort_values('비율').head(10)

,단어,비율
37,bad,-2.688149
392,minutes,-2.619156
939,wasn,-2.465005
65,bland,-2.377994
703,slow,-2.177323
847,took,-2.177323
546,probably,-2.177323
179,don,-1.972529
816,terrible,-1.926009
607,rude,-1.926009


로지스틱 회귀분석

In [11]:
from sklearn.linear_model import LogisticRegressionCV

엘라스틱넷으로 C는 0.001, 0.01, 0.1 세 가지, L1의 비율은 0, 0.5, 1 세 가지를 시도 총 9가지 조합을 시도하여 성능이 가장 좋은 조합을 찾음

In [12]:
model = LogisticRegressionCV(
    penalty='elasticnet', solver='saga', random_state=42,
    Cs=[0.001, 0.01, 0.1], l1_ratios=[0, 0.5, 1])

In [13]:
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=[0.001, 0.01, 0.1], l1_ratios=[0, 0.5, 1],
                     penalty='elasticnet', random_state=42, solver='saga')

가장 좋은 C

In [14]:
model.C_

array([0.1])

가장 좋은 L1의 비율

In [15]:
model.l1_ratio_

array([0])

In [16]:
#훈련 데이터에서 정확도
model.score(x_train, y_train)

0.86375

In [17]:
#테스트 데이터에서 정확도
model.score(x_test, y_test)

0.71

In [18]:
#단어별 가중치
word_coef = pd.DataFrame({
    '단어': cv.get_feature_names_out(),
    '가중치': model.coef_.flat
})

In [19]:
#긍정 단어
word_coef.sort_values('가중치').tail(10)

,단어,가중치
208,excellent,0.335977
56,best,0.352288
32,awesome,0.416480
364,love,0.425192
250,friendly,0.443511
409,nice,0.494441
12,amazing,0.523192
153,delicious,0.720713
265,good,0.801402
268,great,1.215942


In [20]:
#부정 단어
word_coef.sort_values('가중치').head(10)

,단어,가중치
179,don,-0.507271
37,bad,-0.456790
392,minutes,-0.393614
939,wasn,-0.357123
65,bland,-0.335242
703,slow,-0.298549
546,probably,-0.295877
977,worst,-0.291399
928,waited,-0.261590
57,better,-0.251174


In [21]:
#예측
y_pred = model.predict(x_test)

In [22]:
#확률로 예측
probs = model.predict_proba(x_test)

In [23]:
#긍정 확률만
prob = probs[:, 1]

In [24]:
#문턱값에 따라 다르게 예측
threshold = 0.5 # 문턱값
y_pred = np.where(prob > threshold, 1, 0)